In [1]:
%config IPCompleter.use_jedi = False

In [2]:
import numpy as np
np.set_printoptions(suppress = True)
import pandas as pd

import matplotlib.pyplot as plt

# Importing the Dataset
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

from aif360.metrics.utils import compute_boolean_conditioning_vector
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn

import pickle

In [45]:
priv_group = [{'sex': 1}]
unpriv_group = [{'sex': 0}]

In [46]:
data_adult = load_preproc_data_adult(['sex'])

In [47]:
dset_raw_trn, dset_raw_tst = data_adult.split([0.7], shuffle=True)

In [356]:
dset_trn_pred = dset_raw_trn.copy(deepcopy=True)
dset_trn_pred.features = torch.from_numpy(dset_trn_pred.features).float()
dset_trn_pred.labels = torch.from_numpy(dset_trn_pred.labels)

In [206]:
dset_raw_trn_metrics = BinaryLabelDatasetMetric(dset_raw_trn,
                                      unprivileged_groups=unpriv_group,
                                     privileged_groups=priv_group)
print("Statistical Parity Difference:",dset_raw_trn_metrics.statistical_parity_difference())
print("Disparate Impact:",dset_raw_trn_metrics.disparate_impact())
print("Consistency:",dset_raw_trn_metrics.consistency()[0])

Statistical Parity Difference: -0.19334966296595607
Disparate Impact: 0.3633812161549577
Consistency: 0.7172599374067912


In [357]:
# For encoding the p features
class Encoder(nn.Module):
    def __init__(self, size_in, size_out):
        super().__init__()
        self.linear1 = nn.Linear(size_in, size_out)
    
    def forward(self, x):
        x = self.linear1(x)
        return x

In [358]:
# For encoding the p features
class Encoder(nn.Module):
    def __init__(self, size_in, size_out):
        super().__init__()
        self.linear1 = nn.Linear(size_in, size_out)
        self.act1 = nn.Sigmoid()
    
    def forward(self, x):
        x = self.act1(self.linear1(x))
        return x

In [359]:
# For decoding back the above encoded features into p features
class Decoder(nn.Module):
    def __init__(self, size_in, size_out):
        super().__init__()
        self.linear1 = nn.Linear(size_in, size_out) # Should be reverse of that of Encoder's
        self.act1 = nn.Sigmoid()
        
    def forward(self, x):
        x = self.act1(self.linear1(x))
        return x

In [360]:
class FairAutoEnc(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.enc = Encoder(input_dim, hidden_dim)
        self.dec = Decoder(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.enc(x)
        x = self.dec(x)
        return x

In [361]:
# Why the dimension of latent space is 1 less than that of feature space?
# We want to kinda disperse the protected variable among other variables
fair_autoenc = FairAutoEnc(len(data_adult.feature_names),
                           len(data_adult.feature_names)-1,
                          len(data_adult.feature_names))

In [362]:
num_epochs = 2000
learning_rate = 1e-3

optimizer = torch.optim.Adam(fair_autoenc.parameters(), lr=learning_rate)

criterion = nn.MSELoss()

In [363]:
# def fnr_score_diff(x_recon):
#     bool_vec1 = torch.logical_and((x_recon[:, 1] >= 0.5),(torch.ravel(dset_trn_pred.labels) == 1))
#     bool_vec2 = torch.logical_and((x_recon[:, 1] < 0.5),(torch.ravel(dset_trn_pred.labels) == 1))
#     NA_pos = torch.sum(bool_vec1)
#     NB_pos = torch.sum(bool_vec2)
#     FNR_A = torch.sum(dset_trn_pred.labels[bool_vec1])
#     FNR_B = torch.sum(dset_trn_pred.labels[bool_vec2])
#     return torch.abs(-FNR_A/NA_pos + FNR_B/NB_pos)

# def fpr_score_diff(x_recon):
#     bool_vec1 = torch.logical_and((x_recon[:, 1] >= 0.5),(torch.ravel(dset_trn_pred.labels) == 0))
#     bool_vec2 = torch.logical_and((x_recon[:, 1] < 0.5),(torch.ravel(dset_trn_pred.labels) == 0))
#     NA_neg = torch.sum(bool_vec1)
#     NB_neg = torch.sum(bool_vec2)
#     FPR_A = torch.sum(dset_trn_pred.labels[bool_vec1])
#     FPR_B = torch.sum(dset_trn_pred.labels[bool_vec2])
#     return torch.abs(-FPR_A/NA_neg + FPR_B/NB_neg)

In [364]:
# def stat_par_diff(x_recon):
#     prob_suc_A = torch.sum(torch.logical_and(torch.ravel(dset_trn_pred.labels) == 1, x_recon[:, 1] <= 0.5))
#     prob_suc_B = torch.sum(torch.logical_and(torch.ravel(dset_trn_pred.labels) == 1, x_recon[:, 1] > 0.5))
#     prob_A = torch.sum(x_recon[:,1] <= 0.5)
#     prob_B = torch.sum(x_recon[:,1] > 0.5)
#     if prob_A.item() == 0:
#         cond_prob_suc_A = prob_A
#     else:
#         cond_prob_suc_A = prob_suc_A/prob_A
#     if prob_B.item() == 0:
#         cond_prob_suc_B = prob_B
#     else:
#         cond_prob_suc_B = prob_suc_B/prob_B
#     return torch.abs(cond_prob_suc_A - cond_prob_suc_B)

# def disp_imp(x_recon):
#     prob_suc_A = torch.sum(torch.logical_and(torch.ravel(dset_trn_pred.labels) == 1, x_recon[:, 1] <= 0.5))
#     prob_suc_B = torch.sum(torch.logical_and(torch.ravel(dset_trn_pred.labels), x_recon[:, 1] > 0.5))
#     prob_A = torch.sum(x_recon[:,1] <= 0.5)
#     prob_B = torch.sum(x_recon[:,1] > 0.5)
#     return torch.abs(1 - ((prob_suc_A/prob_A) / (prob_suc_B/prob_B)))


In [365]:
def stat_par_diff(x_recon):
    prob_suc_A = torch.sum(x_recon[:,1][torch.ravel(dset_trn_pred.labels) == 0])
    prob_suc_B = torch.sum(x_recon[:,1][torch.ravel(dset_trn_pred.labels) == 1])
    prob_A = torch.sum(x_recon[:,1][x_recon[:,1] <= 0.5])
    prob_B = torch.sum(x_recon[:,1][x_recon[:,1] > 0.5])
    if prob_A.item() == 0:
        cond_prob_suc_A = prob_A
    else:
        cond_prob_suc_A = prob_suc_A/prob_A
    if prob_B.item() == 0:
        cond_prob_suc_B = prob_B
    else:
        cond_prob_suc_B = prob_suc_B/prob_B
    return torch.abs(cond_prob_suc_A - cond_prob_suc_B)

In [366]:
def loss_auto(x_old, x_recon, C1=1):
    non_prot_cols = [0]+list(range(2,len(dset_trn_pred.feature_names)))
    return criterion(x_recon[:,non_prot_cols], x_old[:, non_prot_cols]) + C1*stat_par_diff(x_recon)

In [367]:
# Training fair_autoenc
print("Training fair_autoenc:")
fair_autoenc.train()
for epoch in range(num_epochs):
    dset_trn_pred.features = fair_autoenc(torch.from_numpy(dset_raw_trn.features).float())
    loss= loss_auto(torch.from_numpy(dset_raw_trn.features).float(), dset_trn_pred.features)
    
    loss.backward()
    optimizer.step()
    
    optimizer.zero_grad()
    
    if (epoch+1) % 200== 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        
print("Trained fair_autoenc's Performance on Training Data:")        
with torch.no_grad():
    keep = dset_trn_pred.features
    dset_trn_pred.features = (dset_trn_pred.features > 0.55).numpy().astype(float)
    dset_trn_pred.labels = dset_trn_pred.labels.numpy()
    
    x_recon_metrics = BinaryLabelDatasetMetric(dset_trn_pred,
                                        unprivileged_groups=unpriv_group,
                                        privileged_groups=priv_group)
    print("Statistical Parity Difference:",x_recon_metrics.statistical_parity_difference())
    print("Disparate Impact:",x_recon_metrics.disparate_impact())
    # print("Consistency:",x_recon_metrics.consistency()[0])

Training fair_autoenc:
epoch: 200, loss = 0.3552
epoch: 400, loss = 0.3423
epoch: 600, loss = 0.3391
epoch: 800, loss = 0.3374
epoch: 1000, loss = 0.3361
epoch: 1200, loss = 0.3349
epoch: 1400, loss = 0.3337
epoch: 1600, loss = 0.3323
epoch: 1800, loss = 0.3309
epoch: 2000, loss = 0.3294
Trained fair_autoenc's Performance on Training Data:
Statistical Parity Difference: -0.19334966296595607
Disparate Impact: 0.3633812161549577


In [368]:
keep

tensor([[0.8487, 0.5074, 0.0569,  ..., 0.0381, 0.0570, 0.3228],
        [0.8458, 0.5143, 0.0622,  ..., 0.0431, 0.0617, 0.2778],
        [0.8522, 0.5096, 0.0520,  ..., 0.0347, 0.0512, 0.3176],
        ...,
        [0.8217, 0.5026, 0.0566,  ..., 0.0393, 0.0528, 0.3535],
        [0.8290, 0.5058, 0.0596,  ..., 0.0422, 0.0577, 0.2830],
        [0.8396, 0.5155, 0.0572,  ..., 0.0410, 0.0558, 0.2651]],
       grad_fn=<SigmoidBackward0>)

In [341]:
fair_autoenc.eval()

FairAutoEnc(
  (enc): Encoder(
    (linear1): Linear(in_features=18, out_features=17, bias=True)
    (act1): Sigmoid()
  )
  (dec): Decoder(
    (linear1): Linear(in_features=17, out_features=18, bias=True)
    (act1): Sigmoid()
  )
)

In [369]:
scaler = StandardScaler()
fair_dset_trn = fair_autoenc(torch.from_numpy(dset_raw_trn.features).float())
dset_trn = scaler.fit_transform(fair_dset_trn.detach().numpy())
y_trn = dset_raw_trn.labels.ravel()
dset_trn = torch.from_numpy(dset_trn).float()
y_trn = torch.from_numpy(y_trn).float()
y_trn = y_trn.view(y_trn.shape[0], 1)

In [370]:
class Log_Reg(nn.Module):
    def __init__(self, size_in):
        super().__init__()
        self.linear = nn.Linear(size_in, 1)
    def forward(self, x):
        prob_pred = torch.sigmoid(self.linear(x))
        return prob_pred

In [371]:
M = Log_Reg(len(dset_raw_trn.feature_names))

In [372]:
num_epochs = 2000 # Number of Epochs
learning_rate = 0.01 # Learning Rate

# Stochastic Gradient Descent Optimizers
optimizer_M = torch.optim.SGD(M.parameters(), lr= learning_rate)

# Binary Cross Entropy Loss Functions
criterion = nn.BCELoss()

In [373]:
dset_trn_pred = dset_raw_trn.copy(deepcopy=True)

In [374]:
# Training M
print("Training M:")
M.train()
for epoch in range(num_epochs):
    p_pred = M(dset_trn)
    loss= criterion(p_pred, y_trn)
    
    loss.backward()
    optimizer_M.step()
    
    optimizer_M.zero_grad()
    
    if (epoch+1) % 200== 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        
print("Trained M's Performance on Training Data:")        
with torch.no_grad():
    dset_trn_pred.labels = (p_pred > 0.5).numpy().astype(float)
    
    mod_metrics = ClassificationMetric(dset_raw_trn, dset_trn_pred,
                                        unprivileged_groups=unpriv_group,
                                        privileged_groups=priv_group)
    print("Accuracy:", mod_metrics.accuracy())
    print("Predictive Parity Difference:", mod_metrics.positive_predictive_value(False)-mod_metrics.positive_predictive_value(True))
    print("FNR Difference:", mod_metrics.false_negative_rate_difference())
    print("FPR Difference:", mod_metrics.false_positive_rate_difference())
    print("Accuracy Difference:", mod_metrics.error_rate_difference())
    print("Statistical Parity Difference:",mod_metrics.statistical_parity_difference())
    print("Disparate Impact:",mod_metrics.disparate_impact())
    print("Consistency:",mod_metrics.consistency()[0])

Training M:
epoch: 200, loss = 0.5496
epoch: 400, loss = 0.5042
epoch: 600, loss = 0.4821
epoch: 800, loss = 0.4699
epoch: 1000, loss = 0.4625
epoch: 1200, loss = 0.4577
epoch: 1400, loss = 0.4543
epoch: 1600, loss = 0.4518
epoch: 1800, loss = 0.4499
epoch: 2000, loss = 0.4483
Trained M's Performance on Training Data:
Accuracy: 0.802568077451812
Predictive Parity Difference: -0.6474059003051882
FNR Difference: 0.4577100115074798
FPR Difference: -0.10873384364412098
Accuracy Difference: -0.13004691067237806
Statistical Parity Difference: -0.2147225862822193
Disparate Impact: 0.0
Consistency: 0.7172599374067912


In [352]:
mod_metrics = ClassificationMetric(dset_raw_tst, dset_trn_pred,
                                        unprivileged_groups=unpriv_group,
                                        privileged_groups=priv_group)

In [354]:
dset_trn_pred.labels.shape

(34189, 1)

In [355]:
dset_raw_tst.labels.shape

(14653, 1)

In [375]:
fair_dset_trn

tensor([[0.8487, 0.5074, 0.0569,  ..., 0.0381, 0.0570, 0.3229],
        [0.8458, 0.5143, 0.0622,  ..., 0.0430, 0.0617, 0.2778],
        [0.8522, 0.5096, 0.0520,  ..., 0.0347, 0.0512, 0.3176],
        ...,
        [0.8217, 0.5026, 0.0566,  ..., 0.0393, 0.0528, 0.3536],
        [0.8290, 0.5058, 0.0596,  ..., 0.0422, 0.0577, 0.2830],
        [0.8396, 0.5155, 0.0572,  ..., 0.0410, 0.0558, 0.2651]],
       grad_fn=<SigmoidBackward0>)